In [ ]:
!pip list

In [1]:
!nvidia-smi

Wed Mar 24 10:56:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
     |████████████████████████████████| 6.7MB 37.2MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 23.6MB 131kB/s 
     |████████████████████████████████| 194kB 8.4MB/s 
Cloning into 'mmaction2'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 9954 (delta 48), reused 12 (delta 7), pack-reused 9847
Receiv

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
0.12.0
10.1
GCC 7.3


In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2021-03-22 16:20:43--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  9.73MB/s    in 9.7s    

2021-03-22 16:20:54 (9.62 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = '/content/mmaction2/configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = '/content/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Use load_from_local loader


In [ ]:
video = '/content/mmaction2/demo/abc.mp4'
label = '/content/mmaction2/demo/label_map_k400.txt'
results = inference_recognizer(model, video, label)

In [ ]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

climbing tree:  19.883198
trimming trees:  15.299605
picking fruit:  14.21892
climbing a rope:  12.879446
abseiling:  10.896568


In [ ]:
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip 
#0 is climb
#1 is 

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmaction2/configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = '/content/kinetics400_tiny/train/'
cfg.data_root_val = '/content/kinetics400_tiny/val/'
cfg.ann_file_train = '/content/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = '/content/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = '/content/kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = '/content/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = '/content/kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = '/content/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = '/content/kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = '/content/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = '/content/kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = '/content/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 30
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = '/content/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


2021-03-22 17:04:00,350 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
2021-03-22 17:04:00,402 - mmaction - INFO - load checkpoint from /content/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2021-03-22 17:04:00,404 - mmaction - INFO - Use load_from_local loader
2021-03-22 17:04:00,497 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2021-03-22 17:04:00,500 - mmaction - INFO - Start running, host: root@b683a97162b4, work_dir: /content/tutorial_exps
2021-03-22 17:04:00,501 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
/content/mmaction2/mmaction/core/evaluation/

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.1 task/s, elapsed: 2s, ETA:     0s

2021-03-22 17:04:32,072 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:04:32,074 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2021-03-22 17:04:32,074 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:04:32,080 - mmaction - INFO - 
mean_acc	0.7000
2021-03-22 17:04:32,388 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-03-22 17:04:32,389 - mmaction - INFO - Best top1_acc is 0.7000 at 5 epoch.
2021-03-22 17:04:32,391 - mmaction - INFO - Epoch(val) [5][15]	top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2021-03-22 17:04:36,901 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:02:37, time: 0.900, data_time: 0.758, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 0.6695, loss: 0.6695, grad_norm: 11.0199
2021-03-22 17:04:37,731 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:02:29, time: 0.166, data_time: 0.036, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cl

...

2021-03-22 17:06:07,005 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:06:07,006 - mmaction - INFO - 
top1_acc	0.9000
top5_acc	1.0000
2021-03-22 17:06:07,009 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:06:07,011 - mmaction - INFO - 
mean_acc	0.9000
2021-03-22 17:06:07,012 - mmaction - INFO - Epoch(val) [20][15]	top1_acc: 0.9000, top5_acc: 1.0000, mean_class_accuracy: 0.9000
2021-03-22 17:06:11,030 - mmaction - INFO - Epoch [21][5/15]	lr: 7.813e-05, eta: 0:00:57, time: 0.800, data_time: 0.660, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4516, loss: 0.4516, grad_norm: 8.3901
2021-03-22 17:06:12,210 - mmaction - INFO - Epoch [21][10/15]	lr: 7.813e-05, eta: 0:00:55, time: 0.238, data_time: 0.102, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4541, loss: 0.4541, grad_norm: 9.1029
2021-03-22 17:06:12,864 - mmaction - INFO - Epoch [21][15/15]	lr: 7.813e-05, eta: 0:00:52, time: 0.131, data_time: 0.025, memory: 2918, to

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.0 task/s, elapsed: 2s, ETA:     0s

2021-03-22 17:06:38,120 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:06:38,122 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2021-03-22 17:06:38,124 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:06:38,126 - mmaction - INFO - 
mean_acc	1.0000
2021-03-22 17:06:38,437 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_25.pth.
2021-03-22 17:06:38,439 - mmaction - INFO - Best top1_acc is 1.0000 at 25 epoch.
2021-03-22 17:06:38,440 - mmaction - INFO - Epoch(val) [25][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
2021-03-22 17:06:42,530 - mmaction - INFO - Epoch [26][5/15]	lr: 7.813e-05, eta: 0:00:27, time: 0.817, data_time: 0.653, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3757, loss: 0.3757, grad_norm: 7.9625
2021-03-22 17:06:43,756 - mmaction - INFO - Epoch [26][10/15]	lr: 7.813e-05, eta: 0:00:25, time: 0.245, data_time: 0.107, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, los

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.0 task/s, elapsed: 2s, ETA:     0s

2021-03-22 17:07:10,016 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:07:10,017 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2021-03-22 17:07:10,021 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:07:10,022 - mmaction - INFO - 
mean_acc	1.0000
2021-03-22 17:07:10,023 - mmaction - INFO - Epoch(val) [30][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
